Parse lentiMPRA data into format for modeling.



[Data source](https://www.biorxiv.org/content/10.1101/2023.03.05.531189v1.supplementary-material)
- Sequences: Supplementary Table 3
- Target values: Supplementary Table 4 (summary level data, mean column)